In [ ]:
!pip install pyarrow
!pip install tsfel
!pip install imblearn
!pip install lightgbm
!pip install tensorflow
!pip install catboost
!pip install boruta

In [4]:
import os
import pandas as pd
import numpy as np
import warnings
import boto3
import io
import matplotlib.pyplot as plt
import seaborn as sns
import random
from tqdm import tqdm
from sklearn.pipeline import Pipeline



#!pip install openpyxl

warnings.filterwarnings('ignore')
pd.options.display.float_format = '{:.5f}'.format #evita que muestre notacion cientifica
# pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

module_path = os.path.abspath(os.path.join('../../'))
nombre_carpeta = '../../data/2022'
import sys
if module_path not in sys.path:
    sys.path.append(module_path)
from src.preprocessing.preprocessing  import llenar_val_vacios_str,llenar_val_vacios_ciclo,TsfelVars, ExtraVars,ToDummy, TeEncoder, CardinalityReducer
from src.modeling.simple_models import ChangeTrendPercentajeIdentifierWide,ConstantConsumptionClassifierWide
from src.modeling.supervised_models import LGBMModel, NNModel, LSTMNNModel
from src.modeling.feature_selection import feature_selection_by_constant, feature_selection_by_boruta, feature_selection_by_correlation

2023-06-29 04:51:43.081434: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


## CONTENIDO:
* [Objetivo](#objetivo)
* [Info Inicial](#info)
* [Concatenar Archivos](#concatenar-archivos)
* [Eliminar Gratuitos](#eliminar-gratuitos)
* [Descargar CSV](#descargar-csv)
* [Leer CSV concatenado](#leer-csv)
* [Convertir Formato](#convertir-formato)
* [Verificar Valores Nulos](#valores-nulos)
* [Verificar Cantidad de Valores Nulos](#cantidad-nulos)



## OBJETIVO <a class="anchor" id="objetivo"></a>

1. EPMAPS: El objetivo es indentificar las pérdidas no técnicas en el área de agua


## INFO INICIAL <a class="anchor" id="info"></a>

1. Cuanta data hisotrica tenemos? del 01/2010 al 12/2017

2. Periodicidad del consumo? Mensual

3. Es la data Univariate o multivariate ? 

4. Cual es la frequencia en que se realizará la detección de anomalias (en tiempo real, cada hora, semanal, mensual)? TBD

5. En que unidad se supone que debemos hacer la deteccion de anomalias? volumen robado por usuario, medidor con fraude (pendiente definir)....



## PROCESAR ARCHIVOS <a class="anchor" id="procesar-archivos"></a>

**Obtener directorio**

In [3]:
s3 = boto3.resource('s3')
bucket = s3.Bucket('iagua-quito')#iadbprod-ine-tsp-analyticaldata
# prefix_objs = bucket.objects.filter(Prefix="EPMAPS/data/")


In [4]:
def download_s3_folder(bucket_name, s3_folder, local_dir=None):
    """
    Download the contents of a folder directory
    Args:
        bucket_name: the name of the s3 bucket
        s3_folder: the folder path in the s3 bucket
        local_dir: a relative or absolute directory path in the local file system
    """
    bucket = s3.Bucket(bucket_name)
    for obj in bucket.objects.filter(Prefix=s3_folder):
        target = obj.key if local_dir is None \
            else os.path.join(local_dir, os.path.relpath(obj.key, s3_folder))
        if not os.path.exists(os.path.dirname(target)):
            os.makedirs(os.path.dirname(target))
        if obj.key[-1] == '/':
            continue
        bucket.download_file(obj.key, target)
download_s3_folder('iagua-quito','datos/Datos 2022/','../../data/2022')    

#### Concatenar Archivos <a class="anchor" id="concatenar-archivos"></a>

In [ ]:
#Desde el bucket
%%time
print('[INFO]...concatenando archivos')
bucket_path = 's3://iadbprod-ine-tsp-analyticaldata/' 
df = pd.DataFrame()
for obj in bucket.objects.filter(Prefix='EPMAPS/data/H'):
    key = obj.key
    print(bucket_path+key)
    df_csv = pd.read_csv(bucket_path+key, sep='\t')
    df = df.append(pd.concat([df_csv], axis=0, ignore_index=True), ignore_index=True)
print(df.shape)
print('[INFO]...Done!')

In [ ]:
%%time
#Desde la notebook
nombre_carpeta = '../../data/2022'
contenido = os.listdir(nombre_carpeta)
df = pd.DataFrame()
for elemento in contenido:
    ruta_completa = os.path.join(nombre_carpeta, elemento)
    if os.path.isfile(ruta_completa):  # miramos si es fichero
        agregar = False
        if elemento[-4:].lower()=='.csv':
            df_in = pd.read_csv(ruta_completa, sep=';', 
                               encoding='latin-1', warn_bad_lines=True, error_bad_lines=False,
                               lineterminator='\n')
            agregar = True
        elif elemento[-5:].lower()=='.xlsx':
            df_in = pd.read_excel(ruta_completa, engine='openpyxl')
            agregar = True
        if agregar:
            df = df.append(pd.concat([df_in], axis=0, ignore_index=True), ignore_index=True)
            print(ruta_completa)
df.to_pickle(os.path.join(nombre_carpeta, 'df_raw.pickle'))

#### Convertir Formato y Limpiar <a class="anchor" id="convertir-formato"></a>

In [ ]:
%%time
df.columns = df.columns.str.lower()
df.rename(columns={"cuenta contrato": "numcta","feclvcálc":"mesfac","instalación":"instalacion"}, inplace=True)
df['numcta'] = pd.to_numeric(df['numcta'], errors='coerce')
df['instalacion'] = pd.to_numeric(df['instalacion'], errors='coerce')
df = df[(~df['numcta'].isnull()) & (~df['mesfac'].isnull()) & (~df['instalacion'].isnull())]
df['numcta'] = df['numcta'].astype(int)
df['numcta'] = df['numcta'].astype('string[pyarrow]')
df['instalacion'] = df['instalacion'].astype(int)
df['instalacion'] = df['instalacion'].astype('string[pyarrow]')
df['mesfac'] = df['mesfac'].astype('string[pyarrow]')
df = df[ (df['mesfac'].str.startswith('2021')) | (df['mesfac'].str.startswith('2022')) ]

df['year']=df['mesfac'].str[:4]
df['mes']=df['mesfac'].str[5:7]
df['mes'] = pd.to_numeric(df['mes'], errors='coerce')
df = df[~df['mes'].isnull()]
df['mes'] = df['mes'].astype(int)
df = df[(df['mes']>=1) & (df['mes']<=12)]
df['mes'] = df['mes'].astype('string[pyarrow]').str.zfill(2)
df['date']= df['mes'].astype('str')+'-'+df['year'].astype('str')
df.date = pd.to_datetime(df.date)

#df['comp_zona'] = df.ciclo.astype(str) + df.sector.astype(str) + df.ruta.astype(str) + df.manzana.astype(str) + df.secuencia.astype(str)

campos_nulos = ['cta.contr.', 'instal.', 'dpto', 'pue.sumin.', 'obj.conex.', 'diámetro', 'tp.tarifa', 'inst.pral.', 'lect. act.', 'lect. ant.', 'cons.med.', 'cons.fact.', 'cons.prom.', 'tip.factu.', 'notal', 'est.medido', 'clins', 'clag', 'ctasoloalc']
campos_no_req = ['comp_zona','punto suminis.','obj.conexión','medidor','nombre completo','calle','ced.ruc.pasapor','telef.cliente','celular','caractpredio\r','instal.pral.','cl.agrupación','ctas. solo alca','diam. conex. alc.','caractpredio','consumo promedio','nota de lectura','con agua','con alcanta','numerador','fact.discapa']
df = df.drop(columns = campos_nulos + campos_no_req)

campos_float = ['f.aguaprorrat','fact.alcantpror','prorrat. cargof','desc.anciano.ag','desc.anciano.al','desc.disc.agua','desc.disc.alcan','des.socio.agua','desc.soci.alcan','desc.parro.agua','desc.parro.alca','multasxcta','inter. mora','inter. financia','tasanomenclatur','corte','reconexión','instal. algua','instal. alcant','coactiva','otrosservicio','fact. agua','fact. alcantari']
for col in campos_float:
    df[col] = pd.to_numeric(df[col].str.replace(',','.'), errors='coerce',downcast='float')

campos_int = ['piso','departamento','diám. conex. ap.','consumo promedo','lectura actual','lectura anterior','consumo medido','consumo facturado','fact.socioecono','cl.instalación']
for col in campos_int:
    df[col] = pd.to_numeric(df[col].astype('str').str.replace(',','.').str.extract('(\d+)', expand=False), errors='coerce',downcast='unsigned').fillna(0).astype('int32')
    
#Datos categóricos
cat_tipo_tarifa = ['DOM','COM','OFI','PUB','IND','MUN','SAL']
df['tipo de tarifa'] = pd.Series(np.where(df['tipo de tarifa'].isin(cat_tipo_tarifa),df['tipo de tarifa'],None)).astype('category')
cat_estado_medidor = ['En funcionamiento','En almacén Disponible','Robado','A revisar','En almacén no recuperable','En almacén','En almacén Chatarra','En almacén recuperable','Medidor al revés']
df['estado medidor'] = pd.Series(np.where(df['estado medidor'].isin(cat_estado_medidor),df['estado medidor'],None)).astype('category')
cat_tipo_fact = ['Real','Estimada']
df['tipo facturación'] = pd.Series(np.where(df['tipo facturación'].isin(cat_tipo_fact),df['tipo facturación'],None)).astype('category')
cat_cl_inst = [1,2,3,4]
df['cl.instalación'] = pd.Series(np.where(df['cl.instalación'].isin(cat_cl_inst),df['cl.instalación'],None)).astype('category')
df['ciclo'] = df['ciclo'].str.extract(r'([A-Z]0\d{2})', expand=False).astype('category')
df['ramo'] = df['ramo'].str.extract(r'([A-Z]{,3}\_\d{,2}\_\d{,2}|[A-Z]{,3})', expand=False).astype('category')
df['código hidráulico'] = df['código hidráulico'].str.extract(r'(\d{1,3}\_\d{1,3}\_\d{1,3}\_\d{1,3})', expand=False).astype('category')
df['cargo fijo'] = pd.to_numeric(df['cargo fijo'].str.replace(',','.').str.extract('(\d+\.\d+)', expand=False),errors='coerce',downcast='float')
df['cargo fijo'] = pd.Series(np.where(df['cargo fijo'].isin([2.1,0.0]),df['cargo fijo'],None)).astype('category')
df['diám. conex. ap.'] = pd.Series(np.where(df['diám. conex. ap.']<8,df['diám. conex. ap.'],None)).astype('category')

#QUITAR CARACTERES RAROS QUE QUEDARON EN LUGAR DE LA BARRA EN MESFAC
df.loc[ ~df.mesfac.str.contains('/'),'mesfac'] = df[ ~df.mesfac.str.contains('/') ].year + '/' + df[ ~df.mesfac.str.contains('/') ].mes

#Convertir object a string[pyarrow]
campos_object = df[df.columns[df.dtypes == 'object']].dtypes.keys().to_list()
for col in campos_object:
    df[col] = pd.Series(df[col], dtype="string[pyarrow]")

#Cambiar nombres a campos
df.rename(columns={'tipo de tarifa':'categoria','tipo facturación':'metodo_consumo','diám. conex. ap.':'diam_con_ap'}, inplace=True)
df.rename(columns={'lectura actual':'lectura_actual','lectura anterior':'lectura_anterior','consumo medido':'consumo_medido'}, inplace=True)
df.rename(columns={'consumo facturado':'consumo_facturado','consumo promedo':'consumo_promedio','tipo facturación':'tipo_facturacion'}, inplace=True)
df.rename(columns={'estado medidor':'estado_medidor','cl.instalación':'cl_inst','código hidráulico':'cod_hidraulico','fact. agua':'fact_agua'}, inplace=True)
df.rename(columns={'fact. alcantari':'fact_alcantari','cargo fijo':'cargo_fijo','f.aguaprorrat':'fact_agua_pror','fact.alcantpror':'fact_alcan_pror'}, inplace=True)
df.rename(columns={'prorrat. cargof':'pror_cargo_fijo','desc.anciano.ag':'desc_anciano_agua','desc.anciano.al':'desc_anciano_alcan'}, inplace=True)
df.rename(columns={'desc.disc.agua':'desc_disc_agua','desc.disc.alcan':'desc_disc_alcan','des.socio.agua':'desc_socio_agua','desc.soci.alcan':'desc_socio_alcan'}, inplace=True)
df.rename(columns={'desc.parro.agua':'desc_parro_agua','desc.parro.alca':'desc_parro_alcan','válido de':'valido_de','validez a':'validez_a'}, inplace=True)
df.rename(columns={'fact.socioecono':'fact_socioecono','multasxcta':'multas_x_cta','inter. mora':'inter_mora','inter. financia':'inter_financia'}, inplace=True)
df.rename(columns={'tasanomenclatur':'tasa_nomenc','corte':'corte','reconexión':'reconexion','instal. algua':'instal_algua','instal. alcant':'instal_alcan'}, inplace=True)
df.rename(columns={'coactiva':'coactiva','otrosservicio':'otros_servicio'}, inplace=True)

df.to_parquet(os.path.join(nombre_carpeta, 'df_raw2.parquet'))

#CAMPOS DATASET VIEJO
#'estcta':'estado','conact':'consumo','mancon':'tasa_admon','tasalc':'tasa_alcantarillado','valcon':'valor_cons_usd', 'valfac':'valor_facturado_usd'}, inplace=True)
    
#Unique values para campos con menos de 10 valores unique
#df[ df.columns[(df.nunique()<10).tolist()] ].nunique()

#campos_numericos = df[df.columns[(df.dtypes == 'int32') | (df.dtypes == 'int64') | (df.dtypes == 'float32') | (df.dtypes == 'float64')]].dtypes.keys().to_list()

#Creamos Sample de rows cubriendo todos los meses y categorias
df_sample = pd.DataFrame()
for mes in df.mes.unique():
    for cat in df.categoria.unique():
        #Check if cat is not null or empty: cat was returning nan in some cases, this hack worked
        if cat == cat:
            sub_df = df[ (df.mes==mes) & (df.categoria == cat) ]
            sample = sub_df.sample(min(500,sub_df.shape[0]))
            df_sample = pd.concat([df_sample,sample])
            
#Sample de instalaciones (todas las facturaciones para cada instalacion parejo por categoria)
#Remover cuentas con registros duplicados de facturaciones (combinaciones instalacion-mesfac repetidas)
insts_remover = df[['instalacion','mesfac','consumo_medido']].groupby(['instalacion','mesfac']).agg('count').reset_index().query('consumo_medido>1').instalacion.unique().tolist()
df_limpio = df[ ~df['instalacion'].isin(insts_remover) ]
df_sample_inst_comp = pd.DataFrame()
for cat in df_limpio.categoria.unique():
    lista_insts = df_limpio.query(f'categoria=="{cat}"')['instalacion'].drop_duplicates().tolist()
    #Remover instalaciones agregadas en otras categorías
    if not df_sample_inst_comp.empty:
        lista_insts = list( set(lista_insts) - set(df_sample_inst_comp.instalacion.tolist()) )
    lista_insts = random.sample(lista_insts,min(10000,len(lista_insts)))
    sub_df = df_limpio[ df_limpio.instalacion.isin(lista_insts) ]
    df_sample_inst_comp = pd.concat([df_sample_inst_comp,sub_df])
df_sample_inst_comp.to_parquet(os.path.join(nombre_carpeta, 'df_sample_inst_comp.parquet'))

# CREACION DE DATASET DE FRAUDES A PARTIR DE DATA ORIGINAL
df_fraudes = pd.read_excel(os.path.join(nombre_carpeta, 'multas_final_2021_2022.xlsx'),engine='openpyxl')
df_fraudes = df_fraudes[['Cuenta contrato','Instalación','Fe.inic.extrema','Fecha entrada','Fecha fin real','Fe.inicio real','Código novedad','Causa Generación']]
df_fraudes.rename(columns={'Cuenta contrato':'numcta','Instalación':'instalacion','Fe.inic.extrema':'fecha_multa','Fecha entrada':'fecha_entrada'}, inplace=True)
df_fraudes.rename(columns={'Fecha fin real':'fecha_fin_real','Fe.inicio real':'fecha_inicio_real','Código novedad':'codigo_novedad','Causa Generación':'causa_generacion'}, inplace=True)
df_fraudes = df_fraudes[ ~df_fraudes.instalacion.isnull() ]
df_fraudes.instalacion = df_fraudes.instalacion.astype(int)
df_fraudes.instalacion = df_fraudes.instalacion.astype(str)
df_fraudes.fecha_multa=df_fraudes.fecha_multa.astype(str)

df_fraudes['year']=df_fraudes['fecha_multa'].str[:4]
df_fraudes['mes']=df_fraudes['fecha_multa'].str[5:7]
df_fraudes['dia']=df_fraudes['fecha_multa'].str[8:10]
df_fraudes['date']=  df_fraudes['year']+'/'+df_fraudes['mes']
df_fraudes['mesmulta'] = df_fraudes['year']+'/'+df_fraudes['mes']
df_fraudes.date = pd.to_datetime(df_fraudes.date)

df_fraudes.to_parquet(os.path.join(nombre_carpeta, 'df_fraudes_raw.parquet'))

list_inst_dataset = df_fraudes.instalacion.unique().tolist()
df_fraudes_completo = df[ df['instalacion'].isin(list_inst_dataset) ].drop_duplicates(subset=['instalacion','mesfac'])
cols = ['instalacion','mesmulta','codigo_novedad']
df_fraudes_completo_etiquetado = df_fraudes_completo.merge(df_fraudes[cols], how = 'left', left_on = ['instalacion','mesfac'], 
                                                            right_on = ['instalacion','mesmulta'], indicator = True)
df_fraudes_completo_etiquetado['is_fraud'] = np.where(df_fraudes_completo_etiquetado['codigo_novedad'].isnull(),0,1)
# codigo_novedad>0 -> nunique encontró valores no nulos (NaN/None)
facts_duplicados_fraudulentos = df_fraudes_completo_etiquetado.groupby(['instalacion','mesfac']).agg({'codigo_novedad':'nunique','mesmulta':'count'})\
    .reset_index().query('mesmulta>1 & codigo_novedad>0')[['instalacion','mesfac']].values.tolist()
for f in facts_duplicados_fraudulentos:
    df_fraudes_completo_etiquetado.loc[ (df_fraudes_completo_etiquetado['instalacion']==f[0]) & (df_fraudes_completo_etiquetado['mesfac']==f[1]),'is_fraud'] = 1
df_fraudes_completo_etiquetado = df_fraudes_completo_etiquetado.drop_duplicates(subset=['instalacion','mesfac'])
df_fraudes_completo_etiquetado['metodo_consumo'] = np.where(df_fraudes_completo_etiquetado.metodo_consumo=='Real',0,1)
print(f'{df.shape} {df_fraudes.shape} {df_fraudes_completo_etiquetado.shape} {df_fraudes_completo_etiquetado._merge.value_counts()} {df_fraudes_completo_etiquetado.is_fraud.value_counts()}')

## CONTROLAR SI HAY REGISTROS CON MES MULTA NO VACÍO PERO DISTINTO DE MESFAC (INCONSISTENTE)
# df_comp_etiq.query('~mesmulta.isnull() & mesmulta!=mesfac').shape

df_fraudes_completo_etiquetado.to_parquet(os.path.join(nombre_carpeta, 'df_fraudes_completo_etiquetado.parquet'))



In [6]:
# ARMADO DE DATA EN FORMATO WIDE
import pandas as pd
import os
from tqdm import tqdm
nombre_carpeta = '../../data/2022'
df_fraudes_completo_etiquetado = pd.read_parquet(os.path.join(nombre_carpeta, 'df_fraudes_completo_etiquetado.parquet'))

#Tabla fraudes
fecha_fraud_list = df_fraudes_completo_etiquetado[(df_fraudes_completo_etiquetado.is_fraud==1)&(df_fraudes_completo_etiquetado.date>= '2022-01-01')]['date'].astype(str).unique().tolist()
list_df = []
for fecha_fraud in tqdm(fecha_fraud_list, total=len(fecha_fraud_list)):
    df_etiquetado_fraud = df_fraudes_completo_etiquetado[df_fraudes_completo_etiquetado.date<=fecha_fraud].copy()
    ctas_fraud = df_etiquetado_fraud[(df_etiquetado_fraud.date==fecha_fraud)&(df_etiquetado_fraud.is_fraud==1)].instalacion.unique().tolist()
    df_etiquetado_fraud = df_etiquetado_fraud[df_etiquetado_fraud.instalacion.isin(ctas_fraud)]
    date_inicial = str(pd.to_datetime(fecha_fraud)- pd.DateOffset(months = 12))
    df_etiquetado_fraud = df_etiquetado_fraud[df_etiquetado_fraud['date']>=date_inicial]
    # Obtenemos datos categóricos para la serie anterior al fraude, sin incluirlo
    df_previo = df_etiquetado_fraud[ df_etiquetado_fraud['date']<fecha_fraud ]
    df_cant_null = df_previo.groupby(['instalacion']).metodo_consumo.sum().reset_index(name='cant_consumo_est')
    df_cant_estado = df_previo.groupby(['instalacion']).cl_inst.value_counts().unstack().reset_index()
    df_cant_estado.columns = ['instalacion', 'cant_estado_0', 'cant_estado_1', 'cant_estado_2', 'cant_estado_3', 'cant_estado_4']
    df_cant_estado.fillna(0, inplace=True)
    df_categoria = df_previo.groupby(['instalacion']).categoria.apply(list).reset_index()
    df_categoria["cant_categorias"] = df_categoria.apply(lambda x: len(list(set(x["categoria"]))) if isinstance(x["categoria"],list) else 0,axis=1)
    df_categoria["ult_categoria"] = df_categoria.apply(lambda x: x["categoria"][-1],axis=1)
    
    def cat_mas_freq(row):
        serie = pd.Series(row["categoria"])
        if not serie.value_counts().empty:
            return serie.value_counts().idxmax()
        else:
            return ''
    df_categoria["categ_mas_frecuente"] = df_categoria.apply(cat_mas_freq,axis=1)
    def cambios_categ(row):
        serie = pd.Series(row["categoria"])
        if not serie.value_counts().empty:
            return serie.ne(serie.shift().bfill()).astype(int).sum()
        else:
            return 0
    df_categoria["cambios_categoria"] = df_categoria.apply(cambios_categ,axis=1)
    
    cols_ant = [str(x)+'_anterior' for x in range(12,-1,-1)]
    df_etiquetado_fraud = df_etiquetado_fraud.pivot_table(index=['instalacion'], columns=['date'] , values='consumo_medido')
    df_etiquetado_fraud.columns = cols_ant
    df_etiquetado_fraud['date_fiscalizacion'] = fecha_fraud
    df_etiquetado_fraud.reset_index(inplace=True)
    df_etiquetado_fraud = df_etiquetado_fraud.merge(df_cant_null, on='instalacion')
    df_etiquetado_fraud = df_etiquetado_fraud.merge(df_cant_estado, on='instalacion')
    df_etiquetado_fraud['mes']=mes=int(fecha_fraud[5:7])
    df_etiquetado_fraud['bimestre']=(mes-1)//2+1
    df_etiquetado_fraud['trimestre']=(mes-1)//3+1
    df_etiquetado_fraud['cuatrimestre']=(mes-1)//4+1
    df_etiquetado_fraud['semestre']=(mes-1)//6+1
    df_etiquetado_fraud = df_etiquetado_fraud.merge(df_categoria.drop(columns=['categoria']), on='instalacion')
    list_df.append(df_etiquetado_fraud)
df_fraud_wide = pd.concat(list_df)

#Datos de no fraude
fecha_normal_list = df_fraudes_completo_etiquetado[(df_fraudes_completo_etiquetado.is_fraud==0)&(df_fraudes_completo_etiquetado.date>= '2022-01-01')]['date'].astype(str).unique().tolist()
list_df_normal = []
for fecha_normal in tqdm(fecha_normal_list, total=len(fecha_normal_list)):
    df_etiquetado_normal = df_fraudes_completo_etiquetado[df_fraudes_completo_etiquetado.date<=fecha_normal].copy()
    ctas_normal = df_etiquetado_normal[(df_etiquetado_normal.date==fecha_normal)&(df_etiquetado_normal.is_fraud==0)].instalacion.unique().tolist()
    df_etiquetado_normal = df_etiquetado_normal[df_etiquetado_normal.instalacion.isin(ctas_normal)]
    date_inicial = str(pd.to_datetime(fecha_normal)- pd.DateOffset(months = 12))
    df_etiquetado_normal = df_etiquetado_normal[df_etiquetado_normal['date']>=date_inicial]
    
    # Obtenemos datos categóricos para la serie anterior al control, sin incluirlo
    df_previo = df_etiquetado_normal[ df_etiquetado_normal['date']<fecha_normal ]
    
    df_cant_null = df_previo.groupby(['instalacion']).metodo_consumo.sum().reset_index(name='cant_consumo_est')
    df_cant_estado = df_previo.groupby(['instalacion']).cl_inst.value_counts().unstack().reset_index()
    df_cant_estado.columns = ['instalacion', 'cant_estado_0', 'cant_estado_1', 'cant_estado_2', 'cant_estado_3', 'cant_estado_4']
    df_cant_estado.fillna(0, inplace=True)
    
    df_categoria = df_previo.groupby(['instalacion']).categoria.apply(list).reset_index()
    df_categoria["cant_categorias"] = df_categoria.apply(lambda x: len(list(set(x["categoria"]))) if isinstance(x["categoria"],list) else 0,axis=1)
    df_categoria["ult_categoria"] = df_categoria.apply(lambda x: x["categoria"][-1],axis=1)
    
    def cat_mas_freq(row):
        serie = pd.Series(row["categoria"])
        if not serie.value_counts().empty:
            return serie.value_counts().idxmax()
        else:
            return ''
    df_categoria["categ_mas_frecuente"] = df_categoria.apply(cat_mas_freq,axis=1)
    def cambios_categ(row):
        serie = pd.Series(row["categoria"])
        if not serie.value_counts().empty:
            return serie.ne(serie.shift().bfill()).astype(int).sum()
        else:
            return 0
    df_categoria["cambios_categoria"] = df_categoria.apply(cambios_categ,axis=1)
    
    cols_ant = [str(x)+'_anterior' for x in range(12,-1,-1)]
    df_etiquetado_normal = df_etiquetado_normal.pivot_table(index=['instalacion'], columns=['date'] , values='consumo_medido')
    df_etiquetado_normal.columns = cols_ant
    df_etiquetado_normal['date_fiscalizacion'] = fecha_normal
    df_etiquetado_normal.reset_index(inplace=True)
    df_etiquetado_normal = df_etiquetado_normal.merge(df_cant_null, on='instalacion')
    df_etiquetado_normal = df_etiquetado_normal.merge(df_cant_estado, on='instalacion')
    
    df_etiquetado_normal['mes']=mes=int(fecha_normal[5:7])
    df_etiquetado_normal['bimestre']=(mes-1)//2+1
    df_etiquetado_normal['trimestre']=(mes-1)//3+1
    df_etiquetado_normal['cuatrimestre']=(mes-1)//4+1
    df_etiquetado_normal['semestre']=(mes-1)//6+1
    
    df_etiquetado_normal = df_etiquetado_normal.merge(df_categoria.drop(columns=['categoria']), on='instalacion')
    
    list_df_normal.append(df_etiquetado_normal)

df_normal_wide = pd.concat(list_df_normal)
#Sacar fraudes
df_normal_wide=df_normal_wide[~df_normal_wide.instalacion.isin(df_fraud_wide.instalacion.unique())]

#Conservar solamente registros normales de las 
#fechas de fiscalización de los fraudes
fechas_fraude_unicas = df_fraud_wide.date_fiscalizacion.unique().tolist()
df_normal_wide = df_normal_wide[df_normal_wide.date_fiscalizacion.isin(fechas_fraude_unicas)]
df_normal_wide['is_fraud']=0
df_fraud_wide['is_fraud']=1
#Otros filtros
df_fraud_wide = df_fraud_wide[df_fraud_wide.cant_consumo_est<=6]
df_normal_wide = df_normal_wide[df_normal_wide.cant_consumo_est==0]
df_wide_normal_and_fraud = pd.concat([df_normal_wide,df_fraud_wide]).reset_index(drop=True)
#La siguiente línea duplica los registros que tienen más de una categoría.. Hay que controlar eso
#df_wide_normal_and_fraud = df_wide_normal_and_fraud.merge(df_fraudes_completo_etiquetado[['instalacion','categoria']].drop_duplicates(), on='instalacion')
df_wide_normal_and_fraud['id'] = list(range(len(df_wide_normal_and_fraud)))

df_wide_normal_and_fraud.to_parquet(os.path.join(nombre_carpeta, 'data_normal_and_frauds_wide.parquet'))

100%|██████████| 12/12 [06:34<00:00, 32.91s/it]


In [9]:
# PREPROCESADO DE NA'S Y EXTRACCIÓN DE FEATURES
df = pd.read_parquet(os.path.join(nombre_carpeta, 'data_normal_and_frauds_wide.parquet'))
df = df.rename(columns={'is_fraud':'target','id':'index'})
variables_consumo = [x for x in df.columns if '_anterior' in x and x!='0_anterior']#sin 0_anterior (cons actual)#
df.loc[:,['index']+variables_consumo] = llenar_val_vacios_ciclo(df.loc[:,['index']+variables_consumo], 12)
df.loc[:,['index']+variables_consumo] = df.loc[:,['index']+variables_consumo].fillna(0)
pipe_feature_engeniering_consumo = Pipeline(
    [
        ("tsfel vars", TsfelVars(features_names_path=None,num_periodos= 12)),
        ("add vars3",  ExtraVars(num_periodos=3)),
        ("add vars6",  ExtraVars( num_periodos=6)),
        ("add vars12", ExtraVars(num_periodos=12)),
    ]
        )
df_features = pipe_feature_engeniering_consumo.fit_transform(df[['index']+variables_consumo])
cols_fs = ['index']+df_features.columns.tolist()[13:]
df_completo = df.merge(df_features[cols_fs],how='inner',on=['index'],indicator=False)
df_completo.to_parquet(os.path.join(nombre_carpeta, 'data_complete_wide_features.parquet'))

*** Feature extraction started ***



*** Feature extraction finished ***
*** Feature extraction started ***



*** Feature extraction finished ***


In [2]:
%%time
nombre_carpeta = '../../data/2022'
#df = pd.read_parquet(os.path.join(nombre_carpeta, 'df_raw2.parquet'))
#df_sample = pd.read_parquet(os.path.join(nombre_carpeta, 'df_sample.parquet'))
#df_sample_chico = pd.read_parquet(os.path.join(nombre_carpeta, 'df_sample_chico.parquet'))
df_sample_inst_comp = pd.read_parquet(os.path.join(nombre_carpeta, 'df_sample_inst_comp.parquet'))
#Carga de registros de fraude
df_fraudes = pd.read_parquet(os.path.join(nombre_carpeta, 'df_fraudes_raw.parquet'))
df_fraudes_completo_etiquetado = pd.read_parquet(os.path.join(nombre_carpeta, 'df_fraudes_completo_etiquetado.parquet'))
df_wide_normal_and_fraud = pd.read_parquet(os.path.join(nombre_carpeta, 'data_normal_and_frauds_wide.parquet'))
print(f'{df_sample_inst_comp.shape} {df_fraudes.shape} {df_fraudes_completo_etiquetado.shape}')#{df.shape} {df.columns} {df_sample_chico.shape} {df_sample.shape} 

(1655636, 52) (10433, 13) (221509, 56)
CPU times: user 2.55 s, sys: 1.25 s, total: 3.8 s
Wall time: 16.3 s


In [ ]:
#CANTIDAD DE MULTAS POR CICLO
#df_fraudes_completo_etiquetado[['ciclo', 'sector', 'ruta', 'manzana','secuencia', 'piso', 'departamento']].drop_duplicates().shape
ciclos_multas = df_fraudes_completo_etiquetado.query('~codigo_novedad.isnull()')['ciclo']
print(f'{ciclos_multas.shape} {ciclos_multas.nunique()} {ciclos_multas.value_counts()}')

In [ ]:
##### CÓDIGOS DE VISUALIZACIONES
#PAIRPLOT
import seaborn as sns
import matplotlib.pyplot as plt
sns.set_theme(style="darkgrid")
campos_an = ['lectura_anterior','consumo_facturado','fact_agua','fact_alcantari','categoria']
p1=sns.pairplot(df_sample_chico.reset_index()[ campos_an ],hue='categoria')
plt.savefig(os.path.join(nombre_carpeta, 'fig.png'))

#BOXPLOT Y LOG-BOXPLOT
sns.set(rc={'figure.figsize':(10,7)})
df_sample_chico['consumo_medido_log'] = np.log10(df_sample_chico['consumo_medido']+1)
ax = sns.boxplot(data = df_sample_chico, y= 'consumo_medido_log', x = 'categoria',palette='pastel')
plt.savefig(os.path.join(nombre_carpeta, 'fig.png'))

#BARPLOT CONSUMOS MEDIDOS PROMEDIO POR CATEGORIA
df_agrupado = df[ (df['year']=='2022') & (df['mes'].astype('int')>6) ].groupby(['mes','tipo de tarifa'])['consumo medido'].agg('mean').reset_index()
bar_plot = sns.barplot(data = df_agrupado, x = 'mes', y = 'consumo medido', hue = 'tipo de tarifa') #c = df_agrupado['tipo de tarifa'].map(colors)) #kind='bar', ax=ax, 
fig = bar_plot.get_figure()
fig.savefig(f'{nombre_carpeta}/fig.png')

#Promedio de Consumo por mes por cl.instalación
df_agrupado = df_sample.groupby(['mesfac','cl_inst'])['consumo_medido'].agg('mean').reset_index()
sns.set(rc={'figure.figsize':(10,8)})
bar_plot = sns.barplot(data = df_agrupado, x = 'mesfac', y = 'consumo_medido', hue = 'cl_inst')
for item in bar_plot.get_xticklabels():
    item.set_rotation(90)
fig = bar_plot.get_figure()
fig.savefig(f'{nombre_carpeta}/fig.png')

#DISTRIBUCIÓN DE CONSUMO MEDIDO POR CATEGORÍA
sns.displot(data=df_sample_chico.sample(1000), x='consumo_medido', hue='categoria')#, ax=axs[0])
plt.savefig(os.path.join(nombre_carpeta, 'fig.png'))

#HEATMAP CORRELACIÓN ENTRE VARIABLES NUMÉRICAS
campos_an = ['lectura_anterior','consumo_facturado','fact_agua','fact_alcantari','categoria']
sns.heatmap(df_sample_chico[campos_an].corr(),cmap='RdBu_r', annot=True, vmin=-1, vmax=1)
plt.savefig(os.path.join(nombre_carpeta, 'fig.png'))

#HEATMAP DIAM CONEX VS. CATEGORIA
gr = df_sample_chico[['instalacion','diam_con_ap','categoria']].drop_duplicates()
cr = pd.crosstab(gr.diam_con_ap,gr.categoria)#, normalize='all'
bar_plot = sns.heatmap(cr,annot=False)
fig = bar_plot.get_figure()
fig.savefig(f'{nombre_carpeta}/fig.png')

#LINEPLOT SERIES DE TIEMPO
campo = 'instalacion'#'numcta'
list_ctas = random.sample(list(df[campo].unique()),10)
df_gr = df[ df[campo].isin(list_ctas) ][[campo,'mesfac','consumo_medido']].pivot(index=campo,columns='mesfac',values='consumo_medido').reset_index()
sns.lineplot(data=df_gr[df_gr.columns[1:]].transpose(),legend=False)

#MEDIAS MÓVILES DE CONSUMO MEDIDO POR PERÍODO Y CATEGORÍA
df_sample_filtrado = pd.DataFrame()
for cat in df_sample_inst_comp.categoria.unique():
    if cat == cat:
        lista_insts = random.sample(list(df_sample_inst_comp.query(f'categoria == "{cat}"').instalacion.unique()),10)
        sub_df = df_sample_inst_comp[ df_sample_inst_comp.instalacion.isin(lista_insts) ]
        df_sample_filtrado = pd.concat([df_sample_filtrado,sub_df])
df_sample_filtrado = df_sample_filtrado.sort_values('mesfac')
sns.set(rc={'figure.figsize':(10,8)})
bar_plot = sns.lineplot(data=df_sample_filtrado,x='mesfac',y='consumo_medido',hue='categoria',legend=True)
for item in bar_plot.get_xticklabels():
    item.set_rotation(90)
fig = bar_plot.get_figure()
fig.savefig(f'{nombre_carpeta}/fig.png')

#CANTIDAD DE MULTAS POR MES Y CATEGORIA
df_fraudes['mes_multa'] = df_fraudes['Fe.inic.extrema'].astype('str').str[:7]
df_fraudes_fraude = df_fraudes[ ~df_fraudes['Código novedad'].isnull() ].groupby(['mes_multa','Tipo.Tarifa']).size().reset_index(name='Cantidad')
sns.set(rc={'figure.figsize':(16,12)})
bar_plot = sns.barplot(data = df_fraudes_fraude, x = 'mes_multa', y = 'Cantidad', hue = 'Tipo.Tarifa')
for item in bar_plot.get_xticklabels():
    item.set_rotation(90)
fig = bar_plot.get_figure()
fig.savefig(f'{nombre_carpeta}/fig.png')

#Diferente consumo medido y facturado
#(AGRUPAR PARA VER SI EL TIPO DE FACTURACIÓN ES ESTIMADA, VERSUS REAL CUANDO SON IGUALES)
cons_diferentes = df[ df['consumo_medido'] != df['consumo_facturado'] ].groupby(['categoria','mesfac']).size().reset_index(name='Cantidad')
cons_diferentes['categoria'] = cons_diferentes['categoria'].cat.remove_categories(["IND","PUB","SAL","MUN","OFI"])
cons_diferentes = cons_diferentes[ ~cons_diferentes['categoria'].isnull() ]
sns.set(rc={'figure.figsize':(8,4)})
bar_plot = sns.barplot(data = cons_diferentes, x = 'mesfac', y = 'Cantidad', hue = 'categoria')
for item in bar_plot.get_xticklabels():
    item.set_rotation(90)
fig = bar_plot.get_figure()
fig.savefig(f'{nombre_carpeta}/fig.png')

#Para verificar por grupos la cantidadde NAs
#df[['year','mes','tipo de tarifa','consumo medido']].drop(columns = ['year','mes','tipo de tarifa']).isna().groupby(df[['year','mes','tipo de tarifa']]).sum().reset_index()
#Verificar que columnas tienen valores null
#df.columns[df.isna().any()].tolist()
#df.isnull().sum()
#df_year_completo = (df.groupby(['numcta', 'year'])['mes'].size()).reset_index(name='Count')#.head(50) #ningún año tiene las fechas completas

#VERIFICAR CATEGORIAS ASIGNADAS A CUENTAS O INSTALACIONES (CANTIDAD POR CADA UNA)
campo = 'instalacion'#'numcta'
list_ctas = random.sample(list(df[campo].unique()),100000)
df_gr = df[ df[campo].isin(list_ctas) ][['numcta','instalacion','categoria','cl_inst','mesfac','consumo_medido']]\
    .groupby(campo).agg('nunique').reset_index()#.query('categoria>1')
print(df_gr.shape)
print(df_gr.head())
print('----')
print(f'Distinct CL_INST: {df_gr.cl_inst.value_counts()}')
print('----')
print(f'Distinct CATEGORIA: {df_gr.categoria.value_counts()}')

#CUENTA DE INSTALACIONES POR CATEGORIA Y CL_INST
df_sample_inst_comp[['instalacion','cl_inst','categoria','numcta']].drop_duplicates().groupby(['cl_inst','categoria'])[['instalacion']].count().unstack().head(10)